# Implementation of ALAP Comparison Trials for C Invertible


In [1]:
using Test
using JuMP
using Gurobi
using LinearAlgebra

In [63]:
include("./mpc_functions.jl")
include("./mpc_functions_tests.jl")
include("./linear_system.jl")
include("./linear_system_test.jl")

Test Summary:                | Pass  Total
compute_Sx0() Function Tests |    1      1
Test Summary:               | Pass  Total
compute_Sw() Function Tests |    2      2
Test Summary:                | Pass  Total
compute_C_M() Function Tests |    1      1
Test Summary:             | Pass  Total
LinearSystem Class Struct |    1      1
Test Summary: | Pass  Total
check() tests |    3      3
Test Summary: | Pass  Total
x_dim() Tests |    2      2
Test Summary: | Pass  Total
y_dim() Tests |    4      4
[0.1; 0.1; 0.5; 0.5; 0.2; 0.2]
[0.1, 0.1, 0.5, 0.5, 0.2, 0.2]
Test Summary:                       | Pass  Total
define_simple_eta_HPolytope() Tests |    2      2
Test Summary:                         | Pass  Total
find_ALAP_time_from_X0_to_bound Tests |    1      1


Test.DefaultTestSet("find_ALAP_time_from_X0_to_bound Tests", Any[], 1, false)

In [60]:
A = [1 0;
     -0.5 2];
B = [1;0]
C = [0 1]
eta_w = 0.2
eta_v = 0.1

sys1 = LinearSystem(A,B,C,eta_w,eta_v)

T = 3

n_x = x_dim(sys1)
n_y = y_dim(sys1)

model = Model(Gurobi.Optimizer)

#Clear the model?
empty!(model)

#Create the Optimization Variables
@variable(model,Q[1:dim_x*T,1:dim_x*T])
@variable(model,r[1:dim_x*T,1])
@variable(model,Lambda[1:2*dim_x*(T+1),1:T*(dim_x+dim_y)+n_x0])
@variable(model,alpha0[1:T+1,1])

# compute H_eta and h_eta
H_eta=[kron(Matrix(1I,T,T), H_w) zeros(T*n_w,T*n_y + n_x) ;
    zeros(T*n_v, T*n_x) kron(Matrix(1I,T,T), H_v) zeros(T*n_v,n_x);
    zeros(n_x0,T*(n_x+n_y)) H_x0]

h_eta=[kron(ones(T),h_w); kron(ones(T),h_v); h_x0];

#Constraints
@constraint(model,alpha0.>=0)
@constraint(model,Lambda.>=zeros(size(Lambda)))

S = compute_Sw( system_in.A , T )
C_bar = compute_C_M( system_in.C , [T-1] , T )
J = compute_Sx0( system_in.A , T)

P_xi_w = S + S*Q*C_bar*S
P_xi_v = S * Q 
P_xi_xi0 = ( I + S*Q*C_bar)*J

R_T_mT = [ I ; -Matrix(1I, n_x*(T+1), n_x*(T+1)) ]

# Add dual equality constraint
LHS1 = Lambda[schedule_idx,:,:] * H_eta
RHS1 = R_T_mT * [ P_xi_w P_xi_v P_xi_xi0 ]

@constraint(model, LHS1 .== RHS1 )

# Add dual inequality constraint
alpha_bar=kron(alpha0,ones(n_x(sys1))')
alpha_bar=[alpha_bar alpha_bar];

LHS2 = Lambda * h_eta    
RHS2 = alpha_bar[schedule_idx,:] - R_T_mT * S*r
@constraint(model, LHS2.<=RHS2 )

# low diagonal constraint
for blk_row_idx = 1:div(size(Q_in,1), n_x)-1
    LHS_bloc = Q[(blk_row_idx-1)*n_x.+(1:n_x),blk_row_idx*n_y+1:end]
    @constraint(model,LHS_bloc.==zeros(size(LHS_bloc)))
end

#Optimize!
optimize!(model)

print("termination_status = $(termination_status(model))\n")

Academic license - for non-commercial use only - expires 2020-12-31


LoadError: UndefVarError: n_x0 not defined

In [31]:
temp = size([1])
print("temp =")
print(temp)
print(" with length ")
println(length(temp))

temp = size([1 2; 3 4; 1 2])
print(temp); print(" with length "); println(length(temp))
tempa,tempb = temp
print("a = "); print(tempa); print(", b = "); println(tempb)
print("size([1],1) = "); println(size([1],1))

temp =(1,) with length 1
(3, 2) with length 2
a = 3, b = 2
size([1],1) = 1


In [54]:
print([kron(ones(2,1),2) ; kron(ones(2,1),3) ] )

[2.0; 2.0; 3.0; 3.0]